<a href="https://colab.research.google.com/github/shaja-asm/cry-detection/blob/main/tf_lite_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.utils import Sequence
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
AUDIO_PATH = '/content/drive/MyDrive/CryCorpusFinal'
CRY_FOLDER = os.path.join(AUDIO_PATH, 'cry')
NOTCRY_FOLDER = os.path.join(AUDIO_PATH, 'notcry')
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 25

In [ ]:
def load_audio_files(folder):
    files = []
    for filename in os.listdir(folder):
        if filename.endswith('.wav'):
            files.append(os.path.join(folder, filename))
    return files

def compute_spectrogram(y, sr, n_fft=2048, hop_length=512):
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    D_dB = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    return D_dB

In [ ]:
def save_spectrogram_to_disk(D_dB, save_path):
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    np.save(save_path, D_dB)

cry_files = load_audio_files(CRY_FOLDER)
notcry_files = load_audio_files(NOTCRY_FOLDER)

data = []
labels = []

for idx, file in enumerate(cry_files):
    y, sr = librosa.load(file, sr=None)
    y = librosa.util.normalize(y)
    D_dB = compute_spectrogram(y, sr)
    save_path = os.path.join(f'{0}/spectrograms'.format(AUDIO_PATH), f'cry_{idx}.npy')
    save_spectrogram_to_disk(D_dB, save_path)
    data.append(save_path)
    labels.append(1)

for idx, file in enumerate(notcry_files):
    y, sr = librosa.load(file, sr=None)
    y = librosa.util.normalize(y)
    D_dB = compute_spectrogram(y, sr)
    save_path = os.path.join(f'{0}/spectrograms'.format(AUDIO_PATH), f'notcry_{idx}.npy')
    save_spectrogram_to_disk(D_dB, save_path)
    data.append(save_path)
    labels.append(0)

data = np.array(data)
labels = np.array(labels)



In [ ]:
# Split the datasets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

class OnTheFlyDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_paths, labels, batch_size, img_size, shuffle=True, augment=False):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augment = augment
        self.indices = np.arange(len(self.file_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_file_paths = [self.file_paths[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]

        X, y = self.__data_generation(batch_file_paths, batch_labels)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_file_paths, batch_labels):
        X = np.empty((len(batch_file_paths), *self.img_size, 1), dtype=np.float32)
        y = np.empty((len(batch_file_paths),), dtype=int)

        for i, file_path in enumerate(batch_file_paths):
            D_dB = np.load(file_path)
            D_dB = D_dB[..., np.newaxis]  # Add channel dimension
            D_dB = tf.image.resize(D_dB, self.img_size).numpy()
            if self.augment:
                D_dB = tf.image.random_flip_left_right(D_dB)
                D_dB = tf.image.random_flip_up_down(D_dB)
                D_dB = tf.image.random_brightness(D_dB, max_delta=0.2)
            X[i,] = D_dB
            y[i] = batch_labels[i]

        return X, y

# Augment training data
train_generator = OnTheFlyDataGenerator(X_train, y_train, BATCH_SIZE, IMG_SIZE, shuffle=True, augment=True)
val_generator = OnTheFlyDataGenerator(X_val, y_val, BATCH_SIZE, IMG_SIZE, shuffle=False, augment=False)

# Define the model with batch normalization
model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set up callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch='500,520')

# Save the best model during training
checkpoint_callback = ModelCheckpoint('cry_detection_best_model.keras', monitor='val_loss', save_best_only=True, mode='min')

# Learning rate scheduler
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Train the model with the added callbacks
history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        callbacks=[tensorboard_callback, checkpoint_callback, lr_callback]
    )

print("Training complete. Best model saved as 'cry_detection_best_model.keras'")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


51/51 ━━━━━━━━━━━━━━━━━━━━ 28s 324ms/step - accuracy: 0.7446 - loss: 1.0191 - val_accuracy: 0.4755 - val_loss: 8.6040 - learning_rate: 0.0010
Epoch 2/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 170ms/step - accuracy: 0.8872 - loss: 0.2529 - val_accuracy: 0.4755 - val_loss: 3.4849 - learning_rate: 0.0010
Epoch 3/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 9s 149ms/step - accuracy: 0.9202 - loss: 0.2140 - val_accuracy: 0.4755 - val_loss: 2.5243 - learning_rate: 0.0010
Epoch 4/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 11s 157ms/step - accuracy: 0.9142 - loss: 0.2120 - val_accuracy: 0.6348 - val_loss: 0.9794 - learning_rate: 0.0010
Epoch 5/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - accuracy: 0.9235 - loss: 0.2053 - val_accuracy: 0.6225 - val_loss: 2.9570 - learning_rate: 0.0010
Epoch 6/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 9s 157ms/step - accuracy: 0.9337 - loss: 0.1712 - val_accuracy: 0.7941 - val_loss: 0.8569 - learning_rate: 0.0010
Epoch 7/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 10s 149ms/step - accuracy: 0.9207 - loss: 0.1971 - val_ac

In [ ]:
y_pred = model.predict(val_generator)
y_pred = (y_pred > 0.5).astype(int)
acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f'Accuracy: {acc}')
print(f'F1 Score: {f1}')

model.save('cry_detection_best_model.keras')

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step
Accuracy: 0.821078431372549
F1 Score: 0.7682539682539683


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('cry_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

print('Model converted to TensorFlow Lite and saved.')

Saved artifact at '/tmp/tmp_k8oiezf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135675093283968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675093284496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675093284144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675093284672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675093283792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675093285552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675092518336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675092522384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675092520096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675092522912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135675092517984: